# Part 1: create test documents

Delete previous docs from main collection

In [1]:
import pymongo
mc = pymongo.MongoClient()
db = mc.get_database("Ampel_test")
col = db.get_collection("main")
col.delete_many({})

Instanciate an AlertProcessor without loading default channels

In [2]:
from ampel.pipeline.t0.AlertProcessor import AlertProcessor
from ampel.pipeline.t0.loaders.ZIJSONLoader import ZIJSONLoader
ap = AlertProcessor(load_channels=False, db_name="Ampel_test")

2018-02-22 11:05:56 AlertProcessor.py:54 __init__() INFO Setting up new AlertProcessor instance
2018-02-22 11:05:56 AlertProcessor.py:105 __init__() INFO AlertProcessor initial setup completed


Change Random filter threshold parameter to 1 so that every alert get through filter (Alternatively we could use a NoFilter channel that is currently not set up)<br>
Then load the random channel

In [3]:
ap.channels_config.set_channel_filter_parameter("HU_RANDOM", "threshold", 1)
ap.load_channel("HU_RANDOM")

2018-02-22 11:05:56 AlertProcessor.py:168 __create_channel() INFO Setting up channel: HU_RANDOM
2018-02-22 11:05:56 AlertProcessor.py:177 __create_channel() INFO Loading filter: ampel.contrib.hu.t0.RandFilter
2018-02-22 11:05:56 AlertProcessor.py:185 __create_channel() INFO On match flags: T2RunnableIds.PHOTO_TYPE|AGN|SNCOSMO


Tweak the alert loading function so that we can load self crafted json alerts

In [4]:
ap.set_alert_loading_func(
    ZIJSONLoader.get_flat_pps_list_from_json
)

Run the pipeline with only 5 json alerts

In [5]:
ap.run_iterable_paths(
    base_dir="../alerts/jonas/same_alert_inc_ppts/", 
    extension="*.json", max_entries=5
)

2018-02-22 11:05:56 AlertFileList.py:32 set_folder() DEBUG Target incoming folder: ../alerts/jonas/same_alert_inc_ppts/
2018-02-22 11:05:56 AlertFileList.py:46 set_max_entries() DEBUG Max entries set to: 5
2018-02-22 11:05:56 AlertProcessor.py:267 run_iterable_paths() INFO Returning iterable for file paths in folder: ../alerts/jonas/same_alert_inc_ppts/
2018-02-22 11:05:56 AlertFileList.py:57 build_file_list() DEBUG Building internal file list
2018-02-22 11:05:56 AlertFileList.py:79 build_file_list() DEBUG Filtering files using max_entries criterium
2018-02-22 11:05:56 AlertFileList.py:84 build_file_list() DEBUG File list contains 5 elements
2018-02-22 11:05:56 AlertProcessor.py:286 run() INFO Executing run method
2018-02-22 11:05:56 AlertProcessor.py:320 run() INFO #######     Processing alerts     #######
2018-02-22 11:05:56 AlertProcessor.py:219 load_ingester() INFO Loading ZIAlertIngester
2018-02-22 11:05:56 AlertProcessor.py:366 run() DEBUG Processing: ../alerts/jonas/same_alert_i

Let's see the 5 compound documents that were inserted

In [6]:
from ampel.flags.AlDocTypes import AlDocTypes
from ampel.view.DBDocVizualizer import DBDocVizualizer

vi = DBDocVizualizer()
vi.markup_print_db_entry(
    col.find(
        {"tranId": 1, "alDocType": AlDocTypes.COMPOUND}
    )
)

```json
{
 "_id": "8155bc545f84d9652f1012ef2bdfb6eb",
 "alDocType": COMPOUND,
 "tranId": 1,
 "channels": HU_RANDOM,
 "tier": 0,
 "added": 1519293956.270763,
 "lastppdt": 58037.11874999956,
 "pps": [
  	{"pp": 1000000}
 ]
},
{ "_id": "ee39bebd933038b074bcebdee5c9d0a3",
 "alDocType": COMPOUND,
 "tranId": 1,
 "channels": HU_RANDOM,
 "tier": 0,
 "added": 1519293956.288188,
 "lastppdt": 58049.12083333288,
 "pps": [
  	{"pp": 1000000},
  	{"pp": 1000004}
 ]
},
{ "_id": "14d16ef2f6beb12deb040160a2eb6e29",
 "alDocType": COMPOUND,
 "tranId": 1,
 "channels": HU_RANDOM,
 "tier": 0,
 "added": 1519293956.305275,
 "lastppdt": 58052.11718749956,
 "pps": [
  	{"pp": 1000000},
  	{"pp": 1000004},
  	{"pp": 1000005}
 ]
},
{ "_id": "b8c9c9491718e7b8b1cc1fd4934b39d9",
 "alDocType": COMPOUND,
 "tranId": 1,
 "channels": HU_RANDOM,
 "tier": 0,
 "added": 1519293956.328853,
 "lastppdt": 58055.11197916625,
 "pps": [
  	{"pp": 1000000},
  	{"pp": 1000004},
  	{"pp": 1000005},
  	{"pp": 1000006}
 ]
},
{ "_id": "372147399c47100881a0994fff9478fb",
 "alDocType": COMPOUND,
 "tranId": 1,
 "channels": HU_RANDOM,
 "tier": 0,
 "added": 1519293956.346822,
 "lastppdt": 58058.12083333288,
 "pps": [
  	{"pp": 1000000},
  	{"pp": 1000004},
  	{"pp": 1000005},
  	{"pp": 1000006},
  	{"pp": 1000007}
 ]
}
```

## Part 2: TransientLoader 

soon...

## Part 3: dissect DB query

Let us insert a fake T3 coumpound so that the aggregation query makes more sense to you

In [7]:
fourth_comp = next(
    col.find(
        {"_id": "b8c9c9491718e7b8b1cc1fd4934b39d9"}
    )
)

col.insert_one(
    {
     "_id": "b8c9c9491718e7b8b1cc1faaaaaaaaaa",
     "alDocType": AlDocTypes.COMPOUND,
     "tranId": 1,
     "channels": [5], # HU_RANDOM channel position as of Feb 2018
     "tier": 3,
     "added": fourth_comp['added'] + 1.1,
     "lastppdt": 58055.11197916625,
     "pps": [
          {"pp": 1000000},
          {"pp": 1000004},
          {"pp": 1000005},
          {"pp": 1000006}
     ]
    }
)

This is the full query:

In [8]:
vi.markup_print_db_entry(
    col.aggregate(
       [
          {
             "$match": {
                "tranId": 1, 
                "alDocType": AlDocTypes.COMPOUND
             } 
          }, 
          {
             "$group": {
                "_id": "$tier", 
                "latestAdded": {"$max": "$added"}, 
                "compound": {"$push": "$$ROOT"}
             }
          }, 
          {
             "$sort": {
                "latestAdded": -1
             }
          }, 
          {
             "$limit": 1
          }, 
          {
             "$unwind": "$compound"
          }, 
          {
             "$project": {
                "_id": 0, 
                "compound": 1, 
                "sortValueUsed": {
                   "$cond": { 
                      "if": { 
                         "$eq": ["$compound.tier", 0] 
                      }, 
                      "then": "$compound.lastppdt", 
                      "else": "$compound.added"
                   }
                }
             }
          }, 
          {
             "$sort": {
                "sortValueUsed": -1
             }
          }, 
          {
             "$limit": 1
          },
          { 
             "$replaceRoot": { 
                "newRoot": "$compound" 
             }
          }
        ]
    )
)

```json
{
 "_id": "b8c9c9491718e7b8b1cc1faaaaaaaaaa",
 "alDocType": COMPOUND,
 "tranId": 1,
 "channels": HU_RANDOM,
 "tier": 3,
 "added": 1519293957.4288528,
 "lastppdt": 58055.11197916625,
 "pps": [
  	{"pp": 1000000},
  	{"pp": 1000004},
  	{"pp": 1000005},
  	{"pp": 1000006}
 ]
}
```

Which has returned the latest compound

## Query dissecation
#### Part 1:
match transient ID *and* compound doc type 

In [9]:
vi.markup_print_db_entry(
    col.aggregate(
       [
          {
             "$match": {
                "tranId": 1, 
                "alDocType": AlDocTypes.COMPOUND
             } 
          }
        ]
    )
)

```json
{
 "_id": "8155bc545f84d9652f1012ef2bdfb6eb",
 "alDocType": COMPOUND,
 "tranId": 1,
 "channels": HU_RANDOM,
 "tier": 0,
 "added": 1519293956.270763,
 "lastppdt": 58037.11874999956,
 "pps": [
  	{"pp": 1000000}
 ]
},
{ "_id": "ee39bebd933038b074bcebdee5c9d0a3",
 "alDocType": COMPOUND,
 "tranId": 1,
 "channels": HU_RANDOM,
 "tier": 0,
 "added": 1519293956.288188,
 "lastppdt": 58049.12083333288,
 "pps": [
  	{"pp": 1000000},
  	{"pp": 1000004}
 ]
},
{ "_id": "14d16ef2f6beb12deb040160a2eb6e29",
 "alDocType": COMPOUND,
 "tranId": 1,
 "channels": HU_RANDOM,
 "tier": 0,
 "added": 1519293956.305275,
 "lastppdt": 58052.11718749956,
 "pps": [
  	{"pp": 1000000},
  	{"pp": 1000004},
  	{"pp": 1000005}
 ]
},
{ "_id": "b8c9c9491718e7b8b1cc1fd4934b39d9",
 "alDocType": COMPOUND,
 "tranId": 1,
 "channels": HU_RANDOM,
 "tier": 0,
 "added": 1519293956.328853,
 "lastppdt": 58055.11197916625,
 "pps": [
  	{"pp": 1000000},
  	{"pp": 1000004},
  	{"pp": 1000005},
  	{"pp": 1000006}
 ]
},
{ "_id": "372147399c47100881a0994fff9478fb",
 "alDocType": COMPOUND,
 "tranId": 1,
 "channels": HU_RANDOM,
 "tier": 0,
 "added": 1519293956.346822,
 "lastppdt": 58058.12083333288,
 "pps": [
  	{"pp": 1000000},
  	{"pp": 1000004},
  	{"pp": 1000005},
  	{"pp": 1000006},
  	{"pp": 1000007}
 ]
},
{ "_id": "b8c9c9491718e7b8b1cc1faaaaaaaaaa",
 "alDocType": COMPOUND,
 "tranId": 1,
 "channels": HU_RANDOM,
 "tier": 3,
 "added": 1519293957.4288528,
 "lastppdt": 58055.11197916625,
 "pps": [
  	{"pp": 1000000},
  	{"pp": 1000004},
  	{"pp": 1000005},
  	{"pp": 1000006}
 ]
}
```

#### Intermezzo:
For cosmetic purposes, we define two functions that can handle the new kind of structures that are temporarly created in the aggregation framework

In [10]:
def custom_markup_print(db_cursor):

    json_parts = []

    for el in db_cursor:
        bak = el['compound']
        el['compound'] = []
        for doc in bak: 
            el['compound'].append(
                vi.translate_db_entry(doc)
            )
            
        json_parts.append(
            vi.json_to_str(
                el
            )
        )

    vi.markdown_display_json(
        "".join(json_parts)
    )
    
    
def custom_markup_print2(db_cursor):

    json_parts = []

    for el in db_cursor:
        el['compound'] = vi.translate_db_entry(
            el['compound']   
        )
            
        json_parts.append(
            vi.json_to_str(
                el
            )
        )

    vi.markdown_display_json(
        "".join(json_parts)
    )

#### Part 2:
Group results by **Tier**
<hr>
The result will contain *two documents* because we have compounds from *two* different tiers.<br>
Each document contains all the compounds (field name ```compound```) for a given tier (idientified by the field ```_id``` !)<br>
Additionaly, we create the field ```latestAdded``` that contains the value of the latest added date of compounds for the given tier

In [11]:
custom_markup_print(
    col.aggregate(
       [
          {
             "$match": {
                "tranId": 1, 
                "alDocType": AlDocTypes.COMPOUND
             } 
          }, 
          {
             "$group": {
                "_id": "$tier", 
                "latestAdded": {"$max": "$added"}, 
                "compound": {"$push": "$$ROOT"}
             }
          }
        ]
    )
)

```json
{
 "_id": 3,
 "latestAdded": 1519293957.4288528,
 "compound": [
  {
   "_id": "b8c9c9491718e7b8b1cc1faaaaaaaaaa",
   "alDocType": COMPOUND,
   "tranId": 1,
   "channels": HU_RANDOM,
   "tier": 3,
   "added": 1519293957.4288528,
   "lastppdt": 58055.11197916625,
   "pps": [
    	{"pp": 1000000},
    	{"pp": 1000004},
    	{"pp": 1000005},
    	{"pp": 1000006}
   ]
  }
 ]
},
{ "_id": 0,
 "latestAdded": 1519293956.346822,
 "compound": [
  {
   "_id": "8155bc545f84d9652f1012ef2bdfb6eb",
   "alDocType": COMPOUND,
   "tranId": 1,
   "channels": HU_RANDOM,
   "tier": 0,
   "added": 1519293956.270763,
   "lastppdt": 58037.11874999956,
   "pps": [
    	{"pp": 1000000}
   ]
  },
  {
   "_id": "ee39bebd933038b074bcebdee5c9d0a3",
   "alDocType": COMPOUND,
   "tranId": 1,
   "channels": HU_RANDOM,
   "tier": 0,
   "added": 1519293956.288188,
   "lastppdt": 58049.12083333288,
   "pps": [
    	{"pp": 1000000},
    	{"pp": 1000004}
   ]
  },
  {
   "_id": "14d16ef2f6beb12deb040160a2eb6e29",
   "alDocType": COMPOUND,
   "tranId": 1,
   "channels": HU_RANDOM,
   "tier": 0,
   "added": 1519293956.305275,
   "lastppdt": 58052.11718749956,
   "pps": [
    	{"pp": 1000000},
    	{"pp": 1000004},
    	{"pp": 1000005}
   ]
  },
  {
   "_id": "b8c9c9491718e7b8b1cc1fd4934b39d9",
   "alDocType": COMPOUND,
   "tranId": 1,
   "channels": HU_RANDOM,
   "tier": 0,
   "added": 1519293956.328853,
   "lastppdt": 58055.11197916625,
   "pps": [
    	{"pp": 1000000},
    	{"pp": 1000004},
    	{"pp": 1000005},
    	{"pp": 1000006}
   ]
  },
  {
   "_id": "372147399c47100881a0994fff9478fb",
   "alDocType": COMPOUND,
   "tranId": 1,
   "channels": HU_RANDOM,
   "tier": 0,
   "added": 1519293956.346822,
   "lastppdt": 58058.12083333288,
   "pps": [
    	{"pp": 1000000},
    	{"pp": 1000004},
    	{"pp": 1000005},
    	{"pp": 1000006},
    	{"pp": 1000007}
   ]
  }
 ]
}
```

#### Part 3:
Sort by **latestAdded** <br>

In [12]:
custom_markup_print(
    col.aggregate(
       [
          {
             "$match": {
                "tranId": 1, 
                "alDocType": AlDocTypes.COMPOUND
             } 
          }, 
          {
             "$group": {
                "_id": "$tier", 
                "latestAdded": {"$max": "$added"}, 
                "compound": {"$push": "$$ROOT"}
             }
          }, 
          {
             "$sort": {
                "latestAdded": -1
             }
          }
        ]
    )
)

```json
{
 "_id": 3,
 "latestAdded": 1519293957.4288528,
 "compound": [
  {
   "_id": "b8c9c9491718e7b8b1cc1faaaaaaaaaa",
   "alDocType": COMPOUND,
   "tranId": 1,
   "channels": HU_RANDOM,
   "tier": 3,
   "added": 1519293957.4288528,
   "lastppdt": 58055.11197916625,
   "pps": [
    	{"pp": 1000000},
    	{"pp": 1000004},
    	{"pp": 1000005},
    	{"pp": 1000006}
   ]
  }
 ]
},
{ "_id": 0,
 "latestAdded": 1519293956.346822,
 "compound": [
  {
   "_id": "8155bc545f84d9652f1012ef2bdfb6eb",
   "alDocType": COMPOUND,
   "tranId": 1,
   "channels": HU_RANDOM,
   "tier": 0,
   "added": 1519293956.270763,
   "lastppdt": 58037.11874999956,
   "pps": [
    	{"pp": 1000000}
   ]
  },
  {
   "_id": "ee39bebd933038b074bcebdee5c9d0a3",
   "alDocType": COMPOUND,
   "tranId": 1,
   "channels": HU_RANDOM,
   "tier": 0,
   "added": 1519293956.288188,
   "lastppdt": 58049.12083333288,
   "pps": [
    	{"pp": 1000000},
    	{"pp": 1000004}
   ]
  },
  {
   "_id": "14d16ef2f6beb12deb040160a2eb6e29",
   "alDocType": COMPOUND,
   "tranId": 1,
   "channels": HU_RANDOM,
   "tier": 0,
   "added": 1519293956.305275,
   "lastppdt": 58052.11718749956,
   "pps": [
    	{"pp": 1000000},
    	{"pp": 1000004},
    	{"pp": 1000005}
   ]
  },
  {
   "_id": "b8c9c9491718e7b8b1cc1fd4934b39d9",
   "alDocType": COMPOUND,
   "tranId": 1,
   "channels": HU_RANDOM,
   "tier": 0,
   "added": 1519293956.328853,
   "lastppdt": 58055.11197916625,
   "pps": [
    	{"pp": 1000000},
    	{"pp": 1000004},
    	{"pp": 1000005},
    	{"pp": 1000006}
   ]
  },
  {
   "_id": "372147399c47100881a0994fff9478fb",
   "alDocType": COMPOUND,
   "tranId": 1,
   "channels": HU_RANDOM,
   "tier": 0,
   "added": 1519293956.346822,
   "lastppdt": 58058.12083333288,
   "pps": [
    	{"pp": 1000000},
    	{"pp": 1000004},
    	{"pp": 1000005},
    	{"pp": 1000006},
    	{"pp": 1000007}
   ]
  }
 ]
}
```

#### Part 4:
Retain only first group

In [13]:
custom_markup_print(
    col.aggregate(
       [
          {
             "$match": {
                "tranId": 1, 
                "alDocType": AlDocTypes.COMPOUND
             } 
          }, 
          {
             "$group": {
                "_id": "$tier", 
                "latestAdded": {"$max": "$added"}, 
                "compound": {"$push": "$$ROOT"}
             }
          }, 
          {
             "$sort": {
                "latestAdded": -1
             }
          }, 
          {
             "$limit": 1
          }
        ]
    )
)

```json
{
 "_id": 3,
 "latestAdded": 1519293957.4288528,
 "compound": [
  {
   "_id": "b8c9c9491718e7b8b1cc1faaaaaaaaaa",
   "alDocType": COMPOUND,
   "tranId": 1,
   "channels": HU_RANDOM,
   "tier": 3,
   "added": 1519293957.4288528,
   "lastppdt": 58055.11197916625,
   "pps": [
    	{"pp": 1000000},
    	{"pp": 1000004},
    	{"pp": 1000005},
    	{"pp": 1000006}
   ]
  }
 ]
}
```

#### Part 5:
Unwind: convert single document that contains list of componds (field  ```compound```) in a list of documents each containing a single compound

In [14]:
custom_markup_print2(
    col.aggregate(
       [
          {
             "$match": {
                "tranId": 1, 
                "alDocType": AlDocTypes.COMPOUND
             } 
          }, 
          {
             "$group": {
                "_id": "$tier", 
                "latestAdded": {"$max": "$added"}, 
                "compound": {"$push": "$$ROOT"}
             }
          }, 
          {
             "$sort": {
                "latestAdded": -1
             }
          }, 
          {
             "$limit": 1
          }, 
          {
             "$unwind": "$compound"
          }
        ]
    )
)

```json
{
 "_id": 3,
 "latestAdded": 1519293957.4288528,
 "compound": {
  "_id": "b8c9c9491718e7b8b1cc1faaaaaaaaaa",
  "alDocType": COMPOUND,
  "tranId": 1,
  "channels": HU_RANDOM,
  "tier": 3,
  "added": 1519293957.4288528,
  "lastppdt": 58055.11197916625,
  "pps": [
   	{"pp": 1000000},
   	{"pp": 1000004},
   	{"pp": 1000005},
   	{"pp": 1000006}
  ]
 }
}
```

#### Part 6:
Create ```sortValueUsed``` (conditional) field used part 7 for sorting compounds: <br>
- If surviving compounds originate from Tier 0: use latest photopoint date as value <br>
- If surviving compounds originate from other Tiers: use the compound added date as value

In [15]:
custom_markup_print2(
    col.aggregate(
       [
          {
             "$match": {
                "tranId": 1, 
                "alDocType": AlDocTypes.COMPOUND
             } 
          }, 
          {
             "$group": {
                "_id": "$tier", 
                "latestAdded": {"$max": "$added"}, 
                "compound": {"$push": "$$ROOT"}
             }
          }, 
          {
             "$sort": {
                "latestAdded": -1
             }
          }, 
          {
             "$limit": 1
          }, 
          {
             "$unwind": "$compound"
          }, 
          {
             "$project": {
                "_id": 0, 
                "compound": 1, 
                "sortValueUsed": {
                   "$cond": { 
                      "if": { 
                         "$eq": ["$compound.tier", 0] 
                      }, 
                      "then": "$compound.lastppdt", 
                      "else": "$compound.added"
                   }
                }
             }
          }
        ]
    )
)

```json
{
 "compound": {
  "_id": "b8c9c9491718e7b8b1cc1faaaaaaaaaa",
  "alDocType": COMPOUND,
  "tranId": 1,
  "channels": HU_RANDOM,
  "tier": 3,
  "added": 1519293957.4288528,
  "lastppdt": 58055.11197916625,
  "pps": [
   	{"pp": 1000000},
   	{"pp": 1000004},
   	{"pp": 1000005},
   	{"pp": 1000006}
  ]
 },
 "sortValueUsed": 1519293957.4288528
}
```

#### Part 7:
Sort compounds by mean of ```sortValueUsed``` 

In [16]:
custom_markup_print2(
    col.aggregate(
       [
          {
             "$match": {
                "tranId": 1, 
                "alDocType": AlDocTypes.COMPOUND
             } 
          }, 
          {
             "$group": {
                "_id": "$tier", 
                "latestAdded": {"$max": "$added"}, 
                "compound": {"$push": "$$ROOT"}
             }
          }, 
          {
             "$sort": {
                "latestAdded": -1
             }
          }, 
          {
             "$limit": 1
          }, 
          {
             "$unwind": "$compound"
          }, 
          {
             "$project": {
                "_id": 0, 
                "compound": 1, 
                "sortValueUsed": {
                   "$cond": { 
                      "if": { 
                         "$eq": ["$compound.tier", 0] 
                      }, 
                      "then": "$compound.lastppdt", 
                      "else": "$compound.added"
                   }
                }
             }
          }, 
          {
             "$sort": {
                "sortValueUsed": -1
             }
          }
        ]
    )
)

```json
{
 "compound": {
  "_id": "b8c9c9491718e7b8b1cc1faaaaaaaaaa",
  "alDocType": COMPOUND,
  "tranId": 1,
  "channels": HU_RANDOM,
  "tier": 3,
  "added": 1519293957.4288528,
  "lastppdt": 58055.11197916625,
  "pps": [
   	{"pp": 1000000},
   	{"pp": 1000004},
   	{"pp": 1000005},
   	{"pp": 1000006}
  ]
 },
 "sortValueUsed": 1519293957.4288528
}
```

#### Part 8:
Retain only 'latest' compound

In [17]:
custom_markup_print2(
    col.aggregate(
       [
          {
             "$match": {
                "tranId": 1, 
                "alDocType": AlDocTypes.COMPOUND
             } 
          }, 
          {
             "$group": {
                "_id": "$tier", 
                "latestAdded": {"$max": "$added"}, 
                "compound": {"$push": "$$ROOT"}
             }
          }, 
          {
             "$sort": {
                "latestAdded": -1
             }
          }, 
          {
             "$limit": 1
          }, 
          {
             "$unwind": "$compound"
          }, 
          {
             "$project": {
                "_id": 0, 
                "compound": 1, 
                "sortValueUsed": {
                   "$cond": { 
                      "if": { 
                         "$eq": ["$compound.tier", 0] 
                      }, 
                      "then": "$compound.lastppdt", 
                      "else": "$compound.added"
                   }
                }
             }
          }, 
          {
             "$sort": {
                "sortValueUsed": -1
             }
          }, 
          {
             "$limit": 1
          }
        ]
    )
)

```json
{
 "compound": {
  "_id": "b8c9c9491718e7b8b1cc1faaaaaaaaaa",
  "alDocType": COMPOUND,
  "tranId": 1,
  "channels": HU_RANDOM,
  "tier": 3,
  "added": 1519293957.4288528,
  "lastppdt": 58055.11197916625,
  "pps": [
   	{"pp": 1000000},
   	{"pp": 1000004},
   	{"pp": 1000005},
   	{"pp": 1000006}
  ]
 },
 "sortValueUsed": 1519293957.4288528
}
```

#### Part 9:
Final formatting so that it looks exactly like a DB compound document

In [18]:
vi.markup_print_db_entry(
    col.aggregate(
       [
          {
             "$match": {
                "tranId": 1, 
                "alDocType": AlDocTypes.COMPOUND
             } 
          }, 
          {
             "$group": {
                "_id": "$tier", 
                "latestAdded": {"$max": "$added"}, 
                "compound": {"$push": "$$ROOT"}
             }
          }, 
          {
             "$sort": {
                "latestAdded": -1
             }
          }, 
          {
             "$limit": 1
          }, 
          {
             "$unwind": "$compound"
          }, 
          {
             "$project": {
                "_id": 0, 
                "compound": 1, 
                "sortValueUsed": {
                   "$cond": { 
                      "if": { 
                         "$eq": ["$compound.tier", 0] 
                      }, 
                      "then": "$compound.lastppdt", 
                      "else": "$compound.added"
                   }
                }
             }
          }, 
          {
             "$sort": {
                "sortValueUsed": -1
             }
          }, 
          {
             "$limit": 1
          },
          { 
             "$replaceRoot": { 
                "newRoot": "$compound" 
             }
          }
        ]
    )
)

```json
{
 "_id": "b8c9c9491718e7b8b1cc1faaaaaaaaaa",
 "alDocType": COMPOUND,
 "tranId": 1,
 "channels": HU_RANDOM,
 "tier": 3,
 "added": 1519293957.4288528,
 "lastppdt": 58055.11197916625,
 "pps": [
  	{"pp": 1000000},
  	{"pp": 1000004},
  	{"pp": 1000005},
  	{"pp": 1000006}
 ]
}
```